## Pick last timestep of every run of multirun
* This is me doing the same thing I already did in `CompareMultipleRuns.ipynb` but in different approach because I like wasting time

In [6]:
%matplotlib widget
import xarray as xr
import hvplot.xarray
import pyvista as pv
from JulesD3D.processNetCDF import fixCORs, fixMeshGrid, addVectorSum, makeVelocity
from JulesD3D.utils import quickDF, folderPicker
from os import path, walk
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import Markdown as md

In [7]:
parent_folder = '/Volumes/Seagate Expansion Drive/FinalResults'

# parent_folder = '/Users/julesblom/ThesisResults/'
scenario_folder = folderPicker(parent_folder)
scenario_folder

Select(description='Scenario folder:', layout=Layout(height='120px', width='100%'), options=('/Volumes/Seagate…

## ⚠️ Things blow up when you include Run01 in this folder  ¯\_(ツ)_/¯   ⚠️
Should really figure with what params this would work

In [14]:
all_nc_files_in_folder = scenario_folder.value + '*.nc'
scenario = scenario_folder.value[-10:-1] 
scenario

'Slope0.85'

In [15]:
# hack because first dataset is composed slightly differently as restart datasets 
first_ncfilename = '/Users/julesblom/ThesisResults/Slope085_Run01.nc' # '/Users/julesblom/ThesisResults/Slope125_Run01.nc'

first = xr.open_dataset(first_ncfilename)

In [16]:
# chunking makes adding vector sums much faster!
combined = xr.open_mfdataset(all_nc_files_in_folder, combine='nested', concat_dim="time", chunks={'time': 10},
                             data_vars='minimal', coords='minimal', join='left', compat='override')

In [17]:
bottom_stress_attrs = {'long_name': 'Bottom stress', 'units': 'N/m2', 'grid': 'grid', 'location': 'edge1'}
bottom_stres_dims = ('time', 'M', 'N')

combined = fixCORs(combined)
combined = fixMeshGrid(combined, mystery_flag=True)
combined = addVectorSum(combined, 'TAUKSI', 'TAUETA', key="bottom_stress", attrs=bottom_stress_attrs, dims=bottom_stres_dims)
combined['bottom_diff'] = first.DPS.isel(time = 0) - combined.DPS # Make accumulated deposition/erosion DataArray—

● Fixing mesh grid, assuming a uniform grid 
132 x 182 grid


In [42]:
combined = makeVelocity(first)

makeVelocity: depth not in DataSet already, adding it now...


In [19]:
plot_settings = dict(
    height= 600,
    width= 444,
    rasterize= True,
    dynamic= True,
    attr_labels= True,    
)

In [28]:
_, _, files = next(walk(scenario_folder.value))
nr_of_datasets = len(files) - 1 # .DS_Store wrecking things here
nr_time_steps = int(combined.time.size)
steps_per_dataset = int(nr_time_steps/nr_of_datasets)
times = [i for i in range(steps_per_dataset-1, nr_time_steps+1, steps_per_dataset)]

In [29]:
md(f"""
# Times
| Name | Value |
| --- | --- |
| Nr of timesteps total | {nr_time_steps} |
| Nr of datasets | {nr_of_datasets} |
| Timestep per set | {steps_per_dataset} |
""")


# Times
| Name | Value |
| --- | --- |
| Nr of timesteps total | 588 |
| Nr of datasets | 49 |
| Timestep per set | 12 |


# Depocenter/erosion coords
Ignoring channel

In [ ]:
# thickest_points = []
# lowest_point = []

# for time in range(1, only_last_timestep_per_run.time.size):
#     print('time', time)
#     bottom_diff_at_time = only_last_timestep_per_run.bottom_diff.isel(time=time).squeeze()
    
#     thickest_deposit_height = bottom_diff_at_time.max().values    
#     lowest_deposit_height = bottom_diff_at_time.min().values

#     print("thickest:", thickest_deposit_height, "lowest", lowest_deposit_height)
    
#     x_coord_erosion = bottom_diff_at_time.where(bottom_diff_at_time == lowest_deposit_height, drop=True).squeeze().XZ.values
#     y_coord_erosion = bottom_diff_at_time.where(bottom_diff_at_time == lowest_deposit_height, drop=True).squeeze().YZ.values
    
#     x_coord_depocenter = bottom_diff_at_time.where(bottom_diff_at_time == thickest_deposit_height, drop=True).squeeze().XZ.values
#     y_coord_depocenter = bottom_diff_at_time.where(bottom_diff_at_time == thickest_deposit_height, drop=True).squeeze().YZ.values
    
#     thickest_points.append(
#         {"x": x_coord_depocenter,
#          "y": y_coord_depocenter,
#          "height": float(thickest_deposit_height),
#          "run": int(time-1*5)
#         })
# #     lowest_point_coords.append([int(x_coord_erosion),int(y_coord_depocenter) ])

In [ ]:
thickest_points

In [ ]:
lowest_point_coords

# Bottom stress mean

In [30]:
bottom_stress = combined.bottom_stress

In [32]:
filtered_bottom_stress = bottom_stress.where(bottom_stress > 0.1)

In [ ]:
filtered_bottom_stress.hvplot.quadmesh('XZ', 'YZ',
                        cmap='inferno_r',
                        clim=(0,10),
                        clabel=filtered_bottom_stress.attrs['units'],                                                                       
                        **plot_settings
                       ) 

In [ ]:
filtered_bottom_stress

In [ ]:
mean_bottom_stress_per_time = filtered_bottom_stress.groupby('time').mean(...)

In [33]:
max_bottom_stress_per_time = filtered_bottom_stress.groupby('time').max(...)

All-NaN slice encountered


In [37]:
max_bottom_stress_per_time.plot(aspect=1.8, size=5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.ylabel("Bottom Stress (N/m2)")
plt.title(f"{scenario}° Maximum bottom Stress per timestep (N/m2)")


In [ ]:
mean_bottom_stress_per_time.plot(aspect=1.8, size=4, ylim=(0,16))

## Max density

In [ ]:
density = combined.RHO
filtered_density = density.where(density > 1025.5)

In [ ]:
max_densi_per_time = combined.RHO.groupby('time').max(...)
max_densi_per_time.plot()

In [ ]:
filtered_density.groupby('time').mean(...).plot(aspect=1.8, size=4)

In [ ]:
plt.show()

## Acc net flux max

In [35]:
acc_net_sed_flux = combined.DMSEDCUM.combine
filtered_acc_net_sed_flux = acc_net_sed_flux.where(acc_net_sed_flux > 3)


In [39]:
plt.figure()
# filtered_acc_net_sed_flux.groupby('time').max(...).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

All-NaN slice encountered
All-NaN slice encountered


In [43]:
# def thickestPoint(x):
#     thickest_deposit_height = x.max() 

# #     x_coord_depocenter = x.where(x == thickest_deposit_height, drop=True).squeeze().XZ.values
# #     y_coord_depocenter = x.where(x == thickest_deposit_height, drop=True).squeeze().YZ.values
    
#     return thickest_deposit_height

# def lowestPoint(x):
#     lowest_deposit_height = bottom_diff_at_time.min().values
    
#     x_coord_erosio
#     n = bottom_diff_at_time.where(bottom_diff_at_time == lowest_deposit_height, drop=True).squeeze().XZ.values
#     y_coord_erosion = bottom_diff_at_time.where(bottom_diff_at_time == lowest_deposit_height, drop=True).squeeze().YZ.values
    
# combined.bottom_diff.reduce(thickestPoint)

In [ ]:
combined.bottom_diff